In [98]:
import numpy as np
import pandas as pd
import subprocess
import tqdm 
import edlib

In [4]:
from src.io import read_fasta, read_fastq, read_maf, read_from_folder, read_paf

In [5]:
ref_df, fastq_df, maf_df = read_from_folder("data")

In [ ]:
#minimap = subprocess.call("src/utils/minimap.sh", shell=True)

In [6]:
paf_df = read_paf("data/overlap.paf")

In [ ]:
def windows_generation(target_sequence,overlapping_df,wLength,windows):
    for index, row in overlapping_df.iterrows():
        windows_borders=[]
        r=row['q_start']
        windows_borders=[(0,r//wLength)]
        query_sequence=fastq_df[fastq_df['seq_id']==row['t_seq_name']]['sequence'].iloc[0]
        q_start=row['t_start']
        q_end=row['t_end']
        query_sequence=query_sequence[q_start:q_end]
        edit_distance = edlib.align(query=query_sequence, target=target_sequence, mode="NW", task="path")
        niceNW = edlib.getNiceAlignment(edit_distance, query=query_sequence, target=target_sequence)
        target_alignment=niceNW["query_aligned"]
        query_alignment=niceNW["query_aligned"]
        #print(target_alignment)
        #print(query_alignment)
        w_id=0
        for i in range(0,len(target_alignment)):
            if target_alignment[i]!='-':
                r+=1
                if r%wLength==0:
                    w_id=r/wLength
                    windows_borders.append((i,w_id))
        w_id+=1
        windows_borders.append((len(query_alignment),w_id))
        w=windows_borders[0]
        for i in range(1,len(windows_borders)):
            w_next=windows_borders[i]
            windows[w[1]].append(query_alignment[w[0]:w_next[0]].replace("-",""))
            w=w_next
        windows[w[1]].append(query_alignment[int(wLength*w[1]+1):w[0]].replace("-","")) 
        #print(windows_borders)

## Example

In [175]:
target_sequence=fastq_df[fastq_df['seq_id']=="ecoli11_1"]['sequence'].iloc[0]
overlapping_df=paf_df[paf_df["q_seq_name"] == "ecoli11_1"]
length_window=800
l=len(target_sequence)
n_windows=l//length_window
if l%length_window!=0:
    n_windows+=1
windows={}
previous_index=0
for i in range(0,n_windows):
    windows[i]=[target_sequence[previous_index:min(i*length_window,l)]]

windows_generation(target_sequence,overlapping_df,length_window,windows)   

# Dobbiamo distribuire/parallelizzare da qui...

In [101]:
length_window=800

for target_id in tqdm.tqdm(fastq_df.seq_id):
    target_sequence=fastq_df[fastq_df['seq_id']==target_id]['sequence'].iloc[0]
    overlapping_df=paf_df[paf_df["q_seq_name"] == target_id]
    l=len(target_sequence)
    n_windows=l//length_window

    if l%length_window!=0:
    n_windows+=1
    windows={}
    previous_index=0
    for i in range(0,n_windows):
        windows[i]=[target_sequence[previous_index:min(i*length_window,l)]]

    windows_generation(target_sequence,overlapping_df,length_window,windows)   


    #windows_generation(target,paf_df)

SyntaxError: incomplete input (3462977967.py, line 2)

# Old code

In [ ]:
target_start = paf_target["q_start"].min()
target_end   = paf_target["q_end"].max()

max_str_lenght = 500

In [ ]:
for c in range(target_start, target_end, max_str_lenght):
    paf_target_window = paf_target[(paf_target["q_start"] <= c) & (paf_target["q_end"] >= (c + max_str_lenght))]
    for row in paf_target_window.iterrows():
        row = row[1]
        print(row["t_seq_name"], row["t_start"], row["t_end"])
        print(len(fastq_df[fastq_df["seq_id"] == row["t_seq_name"]]["sequence"].values[0][row["t_start"]:row["t_end"]]))
        break
    break

In [ ]:
x

In [ ]:
t_seqs_maf = maf_df[maf_df["seq_id"].isin(t_seqs)]

In [ ]:
paf_df[paf_df["t_seq_name"] == seq_id]

In [ ]:
t_seqs_ref_start = t_seqs_maf["ref_start"].astype(np.int32)
t_seqs_ref_end   = t_seqs_ref_start + t_seqs_maf["ref_lenght"].astype(np.int32)

max_str_lenght = 500
start = t_seqs_ref_start.min()
end   = t_seqs_ref_end.max()
for c in range(start, end, max_str_lenght):
    a = t_seqs_maf[(t_seqs_ref_start <= c) & (t_seqs_ref_end >= (c + max_str_lenght))]
    if len(a) > 1:
        break

In [ ]:
a

In [ ]:
s = ref_df[ref_df["ref_seq_id"] == a["ref_seq_id"][9379]].sequence[0]

In [ ]:
s[c:(c+500)]

In [ ]:
len(a.ref_seq_alignment[9379])

In [ ]:
len(a.seq_alignment[9379])

In [ ]:
## Per tagliare la sequenza
# 1. prendo la stringa di riferimento e la taglio da c a c+max_str_len
# 2. scorro la sequenza e seq_alignment finchè non trovo il carattere iniziale e finale (?!?!)
# 3. taglio!

In [12]:
import pickle

In [13]:
with open("data_times_task2.pickle", 'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [14]:
content

[(10229, 0.12522006034851074), (4059, 0.13492417335510254)]